This is the setup cell

In [1]:
import datetime
import pathlib
import time
import math

import obspy
from obspy.signal import PPSD
import scipy
import matplotlib.pyplot as plt
import matplotlib
#Run this line if you want interactive plots
%matplotlib qt
import numpy as np

import sprit

ModuleNotFoundError: No module named 'obspy'

In [42]:
import datetime
import pytz

# create a datetime object
dt = datetime.datetime(2023, 5, 15, 22, 0, 0)

dt1 = dt + datetime.timedelta(hours=5, minutes=-5)
print(dt1)
tz = 'US/Central'

local_tz = pytz.timezone(tz)

# get the UTC offset as a timedelta object
utc_offset = local_tz.utcoffset().total_seconds()/60/60
utc_offset

2023-05-16 02:55:00


TypeError: DstTzInfo.utcoffset() missing 1 required positional argument: 'dt'

In [2]:
import sprit

sprit.gui()

In [6]:
dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\Test12_AM.RAC84.00.2023-02-15_2132-2200.mseed'
trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake'
#dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\ChampaignCo\Mahomet2023\Data\RawData"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\RAC84"
dPath = r"C:\Users\riley\OneDrive - University of Illinois - Urbana\Data_OneDrive\Seismic\HVSR\Data\UOFI1_AM.RAC84.00.2023-02-15_1704-1734.mseed"

dPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\LakeCalumet\LC_RaspShake\HVSR Site_AM.RAC84.00.2023.046_2023-02-15_1704-1734.mseed"

trimDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\HVSR\InstrumentTest_PoleBarn\RaspShake_PoleBarn\data\processed_data'

params = sprit.input_param(dataPath=dPath, acq_date='2023-02-15', site='PoleBarn1', starttime='17:04', endtime='17:34')

No metadata file specified!
Using default metadata file for Raspberry Shake v.7 contained in repository at
 c:/Users/riley/LocalData/Code/Github/SPRIT/resources/raspshake_metadata.inv


In [7]:
params = sprit.get_metadata(params)
params = sprit.fetch_data(params=params, source='file', trim_dir=trimDir)

c:\Users\riley\LocalData\Code\Github\SPRIT\.venv\lib\site-packages\obspy\core\inventory\network.py:251: UserWarning: Found more than one matching response. Returning first.
  warnings.warn(msg)


In [22]:
stream = params['stream'].copy()
stream

3 Trace(s) in Stream:
AM.RAC84.00.EHE | 2023-04-25T17:11:59.999999Z - 2023-04-25T17:31:59.999999Z | 100.0 Hz, 120001 samples
AM.RAC84.00.EHN | 2023-04-25T17:11:59.999999Z - 2023-04-25T17:31:59.999999Z | 100.0 Hz, 120001 samples
AM.RAC84.00.EHZ | 2023-04-25T17:11:59.999999Z - 2023-04-25T17:31:59.999999Z | 100.0 Hz, 120001 samples

In [10]:
params = sprit.remove_noise(input=params, kind='manual')
params = sprit.remove_noise(input=params, kind='auto')

c:\Users\riley\LocalData\Code\Github\SPRIT\.venv\lib\site-packages\obspy\signal\trigger.py:183: UserWarning: Warning: converting a masked element to nan.
  data = np.ascontiguousarray(data, dtype=np.float64)


In [11]:
params = sprit.generate_ppsds(params=params, stream=params['stream'], ppsd_length=60, 
                              overlap=0.5, period_step_octaves=0.01, remove_outliers=True, outlier_std=1.5)

In [12]:
hvsr_dict = sprit.process_hvsr(params=params, method=4, resample=True, smooth=3, freq_smooth=False, f_smooth_width=10, remove_outlier_curves=True)
hvsr_dict = sprit.check_peaks(hvsr_dict=hvsr_dict)

No frequency smoothing is being applied. This is not recommended for noisy datasets.


In [21]:
sprit.hvplot(hvsr_dict, kind='hvsr p c+ spec', cmap='turbo', peak_plot=False)

In [28]:
reportDF = sprit.print_report(hvsr_dict, format='csv')

In [29]:
reportDF

,Site Name,Acqusition Date,Longitude,Latitide,Elevation,Peak Frequency,Window Length Freq.,Significant Cycles,Low Curve StDev. over time,Peak Freq. Clarity Below,Peak Freq. Clarity Above,Peak Amp. Clarity,Freq. Stability,Peak Stability (freq. std),Peak Stability (amp. std),Peak Passes
0,PoleBarn1,2023-02-15,-88.229053,40.101212,755,2.863,True,True,True,True,True,True,True,False,True,True


In [22]:
hvsr_dict['Best Peak']

{'f0': 2.8634016737292933,
 'A0': 7.118030915960885,
 'f-': '     0.720 ✔',
 'f+': '    11.382 ✔',
 'Sf': 0.17725842209862042,
 'Sa': 0.028491814897743148,
 'Score': 5,
 'Report': {'Lw': '2.863 > 10 / 60  ✔',
  'Nc': '6872.0 > 200  ✔',
  'σ_A(f)': 'σ_A for all freqs 1.432-5.727 < 2  ✔',
  'A(f-)': 'A(0.72): 1.77 < 3.559  ✔',
  'A(f+)': 'A(11.382): 1.741 < 3.559  ✔',
  'A0': '      7.12 > 2.0 ✔',
  'P+': '2.863 within ±5% of 2.863 ✔',
  'P-': '2.846 within ±5% of 2.863 ✔',
  'Sf': '    0.1773 < 0.05 * 2.863  X',
  'Sa': '    0.0285 < 0.20 ✔'},
 'Pass List': {'Window Length Freq.': True,
  'Significant Cycles': True,
  'Low Curve StDev. over time': True,
  'Peak Freq. Clarity Below': True,
  'Peak Freq. Clarity Above': True,
  'Peak Amp. Clarity': True,
  'Freq. Stability': True,
  'Peak Stability (freq. std)': False,
  'Peak Stability (amp. std)': True},
 'Peak Passes': True}